In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [17]:
def import_data(csv): 
    
    df = pd.read_csv(csv)
    df.index = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df = df.drop('date', axis=1)
    
    close_columns = []
    high_columns = []
    low_columns = []
    open_columns = []
    volume_columns = []
    open_int_columns = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    close_df = df[close_columns]
    high_df = df[high_columns]
    low_df = df[low_columns]
    open_df = df[open_columns]
    volume_df = df[volume_columns]
    open_int_df = df[open_int_columns]
    
    return [close_df, high_df, low_df, open_df, volume_df, open_int_df]

def garman_klass_vol(high_df, low_df, close_df, open_df, period=60):
    """
    Estima a volatilidade a partir dos seguintes preços: alta, baixa, abertura e fechamento
    """
    # Calculando parcelas internas da somatoria
    x_hl = (1/2)*(np.log(np.divide(high_df, low_df))) ** 2
    x_co = - (2 * np.log(2) - 1)* (np.log(np.divide(close_df, open_df))**2)
    
    # Somando parcelas calculadas
    x = x_hl + x_co.values
    
    x.columns = [x[0:3] + "gk" for x in x.columns]
    
    # Criando dataframe para atribuir as volatilidades
    gk = x.copy()
    
    # Termo constante fora da somatoria (Considerando vol diaria)
    const = 1/period
    
    # Atribuindo not a number, para os valores iniciais
    gk.iloc[:period,:] = np.nan
    
    # iteração do centro de massa da vol
    for row in range(period, len(high_df)):
        gk.iloc[row] = np.sqrt(const * np.sum(x.iloc[row-period:row,:]))
        
    return gk

def parkinson_vol(high_df, low_df, period=60):
    """
    Estimando a volatilidade a partir dos preço de Alta e de Baixa
    """
    
    # Calculando parcela interna da somatoria
    x = np.log(np.divide(high_df, low_df)) ** 2
    x.columns = [x[0:3] + "pv" for x in x.columns]
    
    # Criando dataframe para atribuir as volatilidades
    pv = x.copy()
    
    # Termo constante fora da somatoria (Considerando vol diaria)
    const = 1 / (4 * period * np.log(2))
    
    # Atribuindo not a number, para os valores iniciais
    pv.iloc[:period,:] = np.nan
        
    # iteração do centro de massa da vol
    for row in range(period, len(high_df)):
        pv.iloc[row] = np.sqrt(const * np.sum(x.iloc[row-period:row,:]))
        
    return pv

def monthly_volume(volume, period=20):
    
    daily_volume = volume.copy()
    
    for row in range(period, len(volume)):
        daily_volume.iloc[row] = volume.iloc[row-period:row,:].cumsum().iloc[-1]
        
    monthly_volume = daily_volume.resample("BM").last().ffill()
    
    return monthly_volume

def train_test(stock):
    train = pd.DataFrame()

    train["Returns Monthly"] = returns_monthly.iloc[:,stock]
    train["EWMA"] = vol_monthly.iloc[:,stock]
    train["Parkinson"] = pv_monthly.iloc[:,stock]
    train["Garman-Klass"] = gk_monthly.iloc[:,stock]
    train["Monthly Volume"] = monthly_volume.iloc[:,stock]

    prices = close_df.iloc[:,stock]

    for lag in [3, 6, 9, 12]:
        train["Momentum " + str(lag)] = prices.pct_change(lag * 21)

    target = train["Returns Monthly"].shift(-1)
    
    features = train.dropna()
    target = target.dropna().loc['2000-07-31':]
    
    return (features, target)

In [18]:
close_df, high_df, low_df, open_df, volume_df, open_int_df = import_data("../data.csv")

# Retornos diários
returns_daily = close_df.pct_change().fillna(0)

# Retornos mensais
returns_monthly = close_df.pct_change(20).fillna(0).resample('BM').last().ffill()

# Vol diária
vol_daily = returns_daily.ewm(adjust=True, com=60, min_periods=0).std().dropna()

# Vol de 261 dias, apenas o último dia de cada mês
vol_monthly = (np.sqrt(261)*vol_daily).resample('BM').last().ffill()

pv_df = parkinson_vol(high_df, low_df)
pv_monthly = (np.sqrt(261)*pv_df).resample('BM').last().ffill()

gk_df = garman_klass_vol(high_df, low_df, close_df, open_df)
gk_monthly = (np.sqrt(261)*gk_df).resample('BM').last().ffill()

monthly_volume = monthly_volume(volume_df)

feature, target = train_test(1)

In [19]:
feature.head()

,Returns Monthly,EWMA,Parkinson,Garman-Klass,Monthly Volume,Momentum 3,Momentum 6,Momentum 9,Momentum 12
date,,,,,,,,,
2000-07-31,-0.013500,0.081672,0.090117,0.087723,135249.0,-0.051498,-0.084389,-0.095780,-0.052628
2000-08-31,-0.031493,0.076585,0.078957,0.080442,137686.0,-0.025209,-0.094248,-0.106080,-0.107423
2000-09-29,0.010879,0.095101,0.092941,0.094567,227772.0,-0.017524,-0.063966,-0.096061,-0.084917
2000-10-31,-0.010391,0.097895,0.104744,0.107019,101142.0,-0.045012,-0.090034,-0.116647,-0.124354
2000-11-30,-0.019273,0.092711,0.111326,0.112521,92974.0,-0.043417,-0.050007,-0.119602,-0.133054


In [68]:
model = Sequential()

model.add(Dense(200, input_shape=(9,), activation="sigmoid"))

model.add(Dense(100, activation="sigmoid"))

model.add(Dense(20, activation="sigmoid"))

model.add(Dense(1))

opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss="binary_crossentropy", optimizer=opt)

In [69]:
model.fit(feature[:180], target[:180], epochs=20)

Epoch 1/20
180/180 [==============================] - 0s 440us/step - loss: 0.0270
Epoch 2/20
180/180 [==============================] - 0s 50us/step - loss: 0.0270
Epoch 3/20
180/180 [==============================] - 0s 48us/step - loss: 0.0270
Epoch 4/20
180/180 [==============================] - 0s 52us/step - loss: 0.0270
Epoch 5/20
180/180 [==============================] - 0s 49us/step - loss: 0.0270
Epoch 6/20
180/180 [==============================] - 0s 45us/step - loss: 0.0270
Epoch 7/20
180/180 [==============================] - 0s 48us/step - loss: 0.0270
Epoch 8/20
180/180 [==============================] - 0s 52us/step - loss: 0.0270
Epoch 9/20
180/180 [==============================] - 0s 50us/step - loss: 0.0270
Epoch 10/20
180/180 [==============================] - 0s 48us/step - loss: 0.0270
Epoch 11/20
180/180 [==============================] - 0s 44us/step - loss: 0.0270
Epoch 12/20
180/180 [==============================] - 0s 46us/step - loss: 0.0270
Epoch 13/20
